In [2]:
import pandas as pd
import numpy as np

In [3]:
#get df from csv
ncaa_df = pd.read_csv('NCAA data/cleaned_ncaa_data.csv')
ncaa_group = ncaa_df.groupby(['game_id','season','team_alias','full_name']).mean()
ncaa_df.head()

FileNotFoundError: File b'NCAA data/cleaned_ncaa_data.csv' does not exist

In [ ]:
#unneccesary simplification of game_id name with dict
game_id_unique = ncaa_df['game_id'].unique()
game_id_replace = np.arange(0, len(game_id_unique), 1)
game_id_dict = dict(zip(game_id_unique, game_id_replace))
ncaa_df['game_id'] = ncaa_df.game_id.map(game_id_dict)
ncaa_df.head(10)

In [ ]:
#remove games that don't have data for both teams
game_group = ncaa_df.groupby('game_id').nunique()
drop = game_group.loc[omg.team_alias < 2, 'game_id'].index.values
dropped = ncaa_df.set_index('game_id').drop(labels=drop, axis = 0)
dropped = dropped.reset_index()

In [ ]:
#unneccesary simplification of game_id name with dict
game_id_unique = dropped['game_id'].unique()
game_id_replace = np.arange(0, len(game_id_unique), 1)
game_id_dict = dict(zip(game_id_unique, game_id_replace))
dropped['game_id'] = dropped.game_id.map(game_id_dict)
dropped.head()

In [ ]:
#summed groupby season, game_id, and team_alias
game_team_group_sum = dropped.groupby(['season','game_id','team_alias']).sum()
game_team_group_sum.head()

In [ ]:
#get df with team_game_score
game_team_group_sum[['team_game_score']] = game_team_group_sum[['points']]
game_team_group_points_sum = game_team_group_sum[['team_game_score', 'assists']]
game_team_group_points_sum = game_team_group_points_sum.reset_index()
game_team_group_points_sum.head(10)

# use ncaa_merged as DataFrame for individual stats

In [4]:
#leftmerge df's by game_id and team_alias
ncaa_merged = dropped.merge(game_team_group_points_sum, on = ['season','game_id', 'team_alias'], how = 'left')
ncaa_merged.head(10)

NameError: name 'dropped' is not defined

# use ncaa_contribution for player_contribution

In [1]:
#get 'player_contribution' column
ncaa_merged['player_contribution (%)'] = ncaa_merged['points']/ncaa_merged['team_game_score']*100
ncaa_contribution = ncaa_merged[['game_id', 'season', 'team_alias','full_name', 'player_contribution (%)']]
ncaa_contribution.head(25)

NameError: name 'ncaa_merged' is not defined

In [621]:
# star_power = top player contribution / avg(next top4)
#iterate through "team_alias" and append name if name changes
team_names = []
game_count = 0
for row in range(len(ncaa_contribution['team_alias'])):
    if row >0:
        if ncaa_contribution.iloc[row, 2] != ncaa_contribution.iloc[row-1, 2] or ncaa_contribution.iloc[row, 0] != ncaa_contribution.iloc[row-1, 0]:
            team_names.append(ncaa_contribution.iloc[row-1, 2])
len(team_names)

47969

In [625]:
team_names.append('SUU')

In [626]:
#for each team grab top 5 contributors
game_number = 0
counter = 0
star_power = []
for team in team_names:
    try:
        top_five = ncaa_contribution.loc[(ncaa_contribution.game_id ==game_number) & (ncaa_contribution.team_alias ==team)].nlargest(5, 'player_contribution (%)')
        top_list = top_five['player_contribution (%)'].tolist()
        star_power.append(top_list[0]/(np.mean(top_list[1:5])))
        counter += 1
        if counter == 2 :
            game_number += 1
            counter = 0
    except IndexError:
        print('indexerror')
print(star_power)

[2.702702702702702, 1.5652173913043477, 1.3333333333333333, 1.7647058823529411, 1.4166666666666667, 2.5641025641025643, 1.6603773584905666, 1.696969696969697, 2.6, 2.0, 2.333333333333333, 1.8181818181818181, 3.2631578947368416, 1.6190476190476195, 2.888888888888889, 5.4399999999999995, 1.3714285714285714, 1.795918367346939, 2.2962962962962963, 1.8245614035087718, 2.4, 3.4444444444444446, 1.3043478260869565, 2.4615384615384617, 2.5142857142857142, 2.4615384615384617, 1.2972972972972974, 1.8095238095238098, 1.4634146341463414, 3.5, 1.3061224489795917, 2.1904761904761902, 1.9090909090909092, 2.5945945945945947, 1.3114754098360655, 2.0000000000000004, 1.2727272727272725, 1.6, 1.9459459459459463, 1.446808510638298, 1.8000000000000003, 1.3333333333333335, 1.6521739130434783, 2.0, 2.0869565217391304, 2.6, 1.6428571428571426, 2.604651162790698, 2.0, 1.9428571428571426, 1.4634146341463414, 1.2972972972972974, 2.188679245283019, 1.7777777777777777, 1.8, 1.6, 1.5609756097560974, 1.888888888888889

In [493]:
#get 'assists/score' column
game_team_group_points_sum['assists/score'] = game_team_group_points_sum['assists']/game_team_group_points_sum['team_game_score']
team_df =  game_team_group_points_sum.drop('assists', 1)
team_df.head(10)

,season,game_id,team_alias,team_game_score,assists/score
0,2013,0,LOU,66.0,0.212121
1,2013,0,MEM,72.0,0.236111
2,2013,1,ARK,61.0,0.180328
3,2013,1,UGA,66.0,0.106061
4,2013,2,HOU,78.0,0.217949
5,2013,2,USF,69.0,0.188406
6,2013,3,FSU,85.0,0.176471
7,2013,3,VCU,67.0,0.104478
8,2013,4,ASU,76.0,0.236842
9,2013,4,STAN,64.0,0.062500


In [630]:
#add star_power stat to team_df
team_df['star_power'] = np.round(star_power,2)
team_df.head(10)

,season,game_id,team_alias,team_game_score,assists/score,differences,w/l_ratio,star_power
0,2013,0,LOU,66.0,0.212121,NaN,0,2.70
1,2013,0,MEM,72.0,0.236111,6.0,0,1.57
2,2013,1,ARK,61.0,0.180328,-11.0,0,1.33
3,2013,1,UGA,66.0,0.106061,5.0,0,1.76
4,2013,2,HOU,78.0,0.217949,12.0,0,1.42
5,2013,2,USF,69.0,0.188406,-9.0,0,2.56
6,2013,3,FSU,85.0,0.176471,16.0,0,1.66
7,2013,3,VCU,67.0,0.104478,-18.0,0,1.70
8,2013,4,ASU,76.0,0.236842,9.0,0,2.60
9,2013,4,STAN,64.0,0.062500,-12.0,0,2.00


In [631]:
#get team_game_score differences to calculate w/l ratio
team_df['differences'] = team_df['team_game_score'].diff()
team_df.head()

,season,game_id,team_alias,team_game_score,assists/score,differences,w/l_ratio,star_power
0,2013,0,LOU,66.0,0.212121,NaN,0,2.70
1,2013,0,MEM,72.0,0.236111,6.0,0,1.57
2,2013,1,ARK,61.0,0.180328,-11.0,0,1.33
3,2013,1,UGA,66.0,0.106061,5.0,0,1.76
4,2013,2,HOU,78.0,0.217949,12.0,0,1.42


In [632]:
# only grab second team for each game
differences = team_df.iloc[1::2, :]
differences = differences[['differences']]
differences.head()

,differences
1,6.0
3,5.0
5,-9.0
7,-18.0
9,-12.0


In [633]:
#do a join of isolated differences
test_df = team_df.join(differences, how='outer', lsuffix = 'trash',rsuffix = '')
test_df.head(10)

,season,game_id,team_alias,team_game_score,assists/score,differencestrash,w/l_ratio,star_power,differences
0,2013,0,LOU,66.0,0.212121,NaN,0,2.70,NaN
1,2013,0,MEM,72.0,0.236111,6.0,0,1.57,6.0
2,2013,1,ARK,61.0,0.180328,-11.0,0,1.33,NaN
3,2013,1,UGA,66.0,0.106061,5.0,0,1.76,5.0
4,2013,2,HOU,78.0,0.217949,12.0,0,1.42,NaN
5,2013,2,USF,69.0,0.188406,-9.0,0,2.56,-9.0
6,2013,3,FSU,85.0,0.176471,16.0,0,1.66,NaN
7,2013,3,VCU,67.0,0.104478,-18.0,0,1.70,-18.0
8,2013,4,ASU,76.0,0.236842,9.0,0,2.60,NaN
9,2013,4,STAN,64.0,0.062500,-12.0,0,2.00,-12.0


In [634]:
#calculate w/l ratio
team_df['w/l_ratio'] = 0
test_df.loc[test_df.differences > 0, 'w/l_ratio'] = 1
test_df.head(10)

,season,game_id,team_alias,team_game_score,assists/score,differencestrash,w/l_ratio,star_power,differences
0,2013,0,LOU,66.0,0.212121,NaN,0,2.70,NaN
1,2013,0,MEM,72.0,0.236111,6.0,1,1.57,6.0
2,2013,1,ARK,61.0,0.180328,-11.0,0,1.33,NaN
3,2013,1,UGA,66.0,0.106061,5.0,1,1.76,5.0
4,2013,2,HOU,78.0,0.217949,12.0,0,1.42,NaN
5,2013,2,USF,69.0,0.188406,-9.0,0,2.56,-9.0
6,2013,3,FSU,85.0,0.176471,16.0,0,1.66,NaN
7,2013,3,VCU,67.0,0.104478,-18.0,0,1.70,-18.0
8,2013,4,ASU,76.0,0.236842,9.0,0,2.60,NaN
9,2013,4,STAN,64.0,0.062500,-12.0,0,2.00,-12.0


# use this for team stats by individual game

In [635]:
#shift up values of differences column by one to run w/l ratio calcs for other team
test_df.loc[:, 'differences'] = test_df.loc[:, 'differences'].shift(-1)
test_df.loc[test_df.differences < 0, 'w/l_ratio'] = 1
test_df.head(20)

,season,game_id,team_alias,team_game_score,assists/score,differencestrash,w/l_ratio,star_power,differences
0,2013,0,LOU,66.0,0.212121,NaN,0,2.70,6.0
1,2013,0,MEM,72.0,0.236111,6.0,1,1.57,NaN
2,2013,1,ARK,61.0,0.180328,-11.0,0,1.33,5.0
3,2013,1,UGA,66.0,0.106061,5.0,1,1.76,NaN
4,2013,2,HOU,78.0,0.217949,12.0,1,1.42,-9.0
5,2013,2,USF,69.0,0.188406,-9.0,0,2.56,NaN
6,2013,3,FSU,85.0,0.176471,16.0,1,1.66,-18.0
7,2013,3,VCU,67.0,0.104478,-18.0,0,1.70,NaN
8,2013,4,ASU,76.0,0.236842,9.0,1,2.60,-12.0
9,2013,4,STAN,64.0,0.062500,-12.0,0,2.00,NaN


In [697]:
with_wins = test_df[['season','game_id','team_alias','team_game_score', 'w/l_ratio']]
with_wins.loc[with_wins['w/l_ratio'] == 0, 'w/l_ratio'] = 'L'
with_wins.loc[with_wins['w/l_ratio'] == 1, 'w/l_ratio'] = 'W'
with_wins.head()

,season,game_id,team_alias,team_game_score,w/l_ratio
0,2013,0,LOU,66.0,L
1,2013,0,MEM,72.0,W
2,2013,1,ARK,61.0,L
3,2013,1,UGA,66.0,W
4,2013,2,HOU,78.0,W


In [653]:
win_count = test_df.groupby(['season','team_alias']).sum()
win_count.head()

game_id  team_game_score  assists/score  differencestrash  \
season team_alias                                                              
2013   AAMU           1409             52.0       0.134615             -12.0   
       AC             5339            214.0       0.723243             -78.0   
       AFA            4818            226.0       0.825461             -36.0   
       AKR            4234            247.0       0.609451             -29.0   
       ALA           36637           2068.0       4.976160            -130.0   

                   w/l_ratio  star_power  differences  
season team_alias                                      
2013   AAMU                0        3.09         36.0  
       AC                  0        7.08        120.0  
       AFA                 0        9.71         78.0  
       AKR                 1        7.34         54.0  
       ALA                12       71.32         17.0

In [654]:
win_count = win_count.reset_index()

In [655]:
win_count['win_count'] = win_count['w/l_ratio']
win_count = win_count[['season','team_alias','win_count']]
win_count.head()

,season,team_alias,win_count
0,2013,AAMU,0
1,2013,AC,0
2,2013,AFA,0
3,2013,AKR,1
4,2013,ALA,12


In [704]:
#make a table of mean team_performance by season and team
team_performance = test_df.groupby(['season','team_alias']).mean()
team_performance=team_performance.reset_index()
team_performance = team_performance.merge(win_count, on=['season','team_alias'],how = 'outer')
team_performance.head()

,season,team_alias,game_id,team_game_score,assists/score,differencestrash,w/l_ratio,star_power,differences,win_count
0,2013,AAMU,1409.00000,52.000000,0.134615,-12.000000,0.000000,3.090000,36.000000,0
1,2013,AC,1334.75000,53.500000,0.180811,-19.500000,0.000000,1.770000,30.000000,0
2,2013,AFA,1204.50000,56.500000,0.206365,-9.000000,0.000000,2.427500,19.500000,0
3,2013,AKR,1058.50000,61.750000,0.152363,-7.250000,0.250000,1.835000,13.500000,1
4,2013,ALA,1181.83871,66.709677,0.160521,-4.193548,0.387097,2.300645,0.548387,12


In [719]:
game_count = test_df.groupby(['season','team_alias']).count()
game_count['game_count'] = game_count['game_id']
game_count = game_count[game_count['game_count'] > 25]
game_count = game_count[['game_count']]
game_count.head()

game_count
season team_alias            
2013   ALA                 31
       ARIZ                38
       ARK                 34
       ASU                 33
       AUB                 30

In [720]:
game_count = game_count.reset_index()

In [723]:
final_df = team_performance.merge(game_count, on = ['season','team_alias'], how = 'outer').dropna()
final_df.to_csv('scatter_ncaa.csv')

In [657]:
team_performance = team_performance.groupby(['season','team_alias']).mean()
team_performance = team_performance[['win_count','w/l_ratio','star_power','assists/score','team_game_score']]
team_performance = team_performance.sort_values(['win_count', 'w/l_ratio','star_power'], ascending = False)
team_performance.head()

,,win_count,w/l_ratio,star_power,assists/score,team_game_score
season,team_alias,,,,,
2014,UK,38,0.974359,1.689487,0.188495,74.358974
2016,GONZ,37,0.948718,1.716667,0.182631,82.615385
2013,FLA,36,0.923077,1.987436,0.184149,69.923077
2014,WIS,36,0.900000,1.937250,0.174833,72.500000
2017,VILL,36,0.900000,1.800250,0.186407,86.575000


In [658]:
team_performance=team_performance.reset_index()

In [663]:
#how to search results by year
top_ten_by_season = team_performance.loc[team_performance.season == 2013].nlargest(10, 'win_count').groupby('season').mean()
top_ten_by_season.head()

,win_count,w/l_ratio,star_power,assists/score,team_game_score
season,,,,,
2013,31.3,0.834215,1.904971,0.186158,73.823413


# avg team_performance by year of top 10 teams

In [729]:
#avg team_performance by year for top 10 teams
season_avg = []
for season in team_performance['season'].unique():
    season_avg.append(team_performance.loc[team_performance.season == season].nlargest(34, 'win_count'))
#concat and sort
top_ten_per_season = pd.concat(season_avg)
top_ten_per_season.head()

,season,team_alias,game_id,team_game_score,assists/score,differencestrash,w/l_ratio,star_power,differences,win_count
87,2013,FLA,1160.000000,69.923077,0.184149,-1.641026,0.923077,1.987436,-14.466667,36
330,2013,WICH,874.057143,74.657143,0.183337,13.800000,0.971429,1.979143,-17.000000,34
11,2013,ARIZ,1012.447368,72.868421,0.205605,5.815789,0.868421,1.689211,-14.315789,33
51,2013,CONN,1065.200000,71.825000,0.168568,4.325000,0.800000,2.144500,-9.200000,32
145,2013,LOU,1074.405405,81.027027,0.189206,15.555556,0.837838,1.941622,-18.478261,31


In [730]:
top_ten_per_season.to_csv('top_ten_pct_ncaa.csv')

In [700]:
top_ten_by_season = top_ten_by_season.reset_index()
top_ten_sem = top_ten_sem.reset_index()

In [701]:
top_ten_by_season =top_ten_by_season.merge(top_ten_sem, on='season', suffixes = ('_mean', '_sem'))

In [702]:
#pivot
top_ten_by_season['stat']=str('star_power')
top_ten_star_power = top_ten_by_season.pivot(index = 'stat',columns = 'season',values = 'star_power_mean')
top_ten_by_season['stat']=str('assists/score')
top_ten_assists_score = top_ten_by_season.pivot(index = 'stat',columns = 'season',values = 'assists/score_mean')
top_ten_by_season['stat']=str('star_power_sem')
top_ten_star_sem = top_ten_by_season.pivot(index = 'stat',columns = 'season',values = 'star_power_sem')
top_ten_by_season['stat']=str('assists/score_sem')
top_ten_assists_sem = top_ten_by_season.pivot(index = 'stat',columns = 'season',values = 'assists/score_sem')
top_ten_pivot = pd.concat([top_ten_star_power, top_ten_assists_score,top_ten_star_sem,top_ten_assists_sem])
top_ten_pivot

season,2013,2014,2015,2016,2017
stat,,,,,
star_power,1.904971,1.797953,1.925370,1.871518,1.920190
assists/score,0.186158,0.190408,0.202837,0.198761,0.201953
star_power_sem,0.055659,0.035086,0.083516,0.046258,0.060888
assists/score_sem,0.006508,0.003210,0.007889,0.006552,0.004924


In [703]:
top_ten_pivot.to_csv('ncaa_top_ten_avg_by_season.csv')